# Getting a list of jobs at several UN organizations using Scrapy

The website unjobs.org lists several positions offered by a wide group of UN and UN-related organizations in countries all over the world. I created a database of the most recently posted jobs with their respective application links and some basic characteristics of each job: organization, city and country. I expect this to be very useful for anyone looking for a job at the international development sector and wants to browse through active posts and filter by employer and location. 

In [1]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
import json
import pandas as pd
pd.set_option('display.max_columns', None)
from bs4 import BeautifulSoup as bs
import requests

Here I first create a pipeline class that writes all found items to a JSON file, where each line contains one JSON element.

The second class is my Spider to parse through several links containing vacancies.

The final three lines start the crawler process.

In [2]:
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('unjobs.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item
    
#Spider to Scrape the links with the vacancies
class MySpider(scrapy.Spider):
    name = "unjobs"
    start_urls = ['https://unjobs.org/new/%d' %i for i in range(1,100)] # this allows to parse through several pages of the site
    custom_settings = {
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': 'unjobs.json'
    }
    #This function extracts the link for each vacancy
    def parse(self, response):
        for job_url in response.css('.job a ::attr("href")').extract():
            yield response.follow(job_url, callback=self.parse_jobs)
    
    #This function goes to every link obtained in 'parse' and extracts the four elements I want to put in my database.
    # It yields a dictionary that is stored in the json file specified above.
    def parse_jobs(self, response):
        title = response.xpath('//td//h2/descendant::text()').extract()
        table = response.xpath("//li[@class='list-group-item dropdown-toggle']//a/text()").extract()
        org = table[0]
        country = table[1]
        city = table[2]
        link = response.url
        yield {
            'job_title': ''.join(title),
            'organization': ''.join(org),
            'country': ''.join(country),
            'city': ''.join(city),
            'link': ''.join(link)
        }

# Starting the Crawler    
process = CrawlerProcess(get_project_settings())
process.crawl(MySpider)
process.start()

2018-11-18 22:51:08 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-11-18 22:51:08 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 18.7.0, Python 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2p  14 Aug 2018), cryptography 2.2.2, Platform Windows-10-10.0.17134-SP0
2018-11-18 22:51:08 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'unjobs.json'}
2018-11-18 22:51:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2018-11-18 22:51:11 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.do

2018-11-18 22:51:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/vacancies/1541546544604> (referer: https://unjobs.org/closing/9)
2018-11-18 22:51:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1541601137803>
{'job_title': 'BRA/06/024 - Contratação de consultor para o Projeto "Elaboração de Informativos em Formato de Vídeos e Cartilha relativos à Matriz de Saldos Contábeis - MSC", Brasília', 'organization': 'UNDP - United Nations Development Programme', 'country': 'Brazil', 'city': 'Brasilia, Brazil', 'link': 'https://unjobs.org/vacancies/1541601137803'}
2018-11-18 22:51:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1541546280421>
{'job_title': 'Intern - Administration (Host Country Services), Nairobi', 'organization': 'UNON - United Nations Office at Nairobi', 'country': 'Kenya', 'city': 'Nairobi, Kenya', 'link': 'https://unjobs.org/vacancies/1541546280421'}
2018-11-18 22:51:16 [scrapy.core.scraper] 

2018-11-18 22:51:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/vacancies/1540591190882> (referer: https://unjobs.org/closing/17)
2018-11-18 22:51:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540832576860>
{'job_title': 'Language Assistant, Sievierodonetsk, Ukraine', 'organization': 'OSCE - Organization for Security and Co-operation in Europe', 'country': 'Ukraine', 'city': 'Sievierodonetsk, Ukraine', 'link': 'https://unjobs.org/vacancies/1540832576860'}
2018-11-18 22:51:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540829439916>
{'job_title': 'Senior ICT Associate, Santo Domingo, Dominican Republic', 'organization': "UNICEF - United Nations Children's Fund", 'country': 'Dominican Republic', 'city': 'Santo Domingo', 'link': 'https://unjobs.org/vacancies/1540829439916'}
2018-11-18 22:51:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540901432284>
{'job_title': 'Chi

2018-11-18 22:51:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540918000149>
{'job_title': 'WHE Team Lead, Tripoli, Libya', 'organization': 'WHO - World Health Organization', 'country': 'Libya', 'city': 'Tripoli, Libya', 'link': 'https://unjobs.org/vacancies/1540918000149'}
2018-11-18 22:51:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540914699478>
{'job_title': 'Team Assistant, Nigeria', 'organization': 'UNIDO - United Nations Industrial Development Organization', 'country': 'Nigeria', 'city': 'Abuja, Nigeria', 'link': 'https://unjobs.org/vacancies/1540914699478'}
2018-11-18 22:51:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/vacancies/1540996449870> (referer: https://unjobs.org/closing/17)
2018-11-18 22:51:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540910896234>
{'job_title': 'Deputy Director General DG/MS2&3, Frankfurt, Germany', 'organization': 'ECB - Eur

2018-11-18 22:51:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/vacancies/1540260986653> (referer: https://unjobs.org/closing/18)
2018-11-18 22:51:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540347873232>
{'job_title': 'Public Information Assistant, Hindi, New York', 'organization': 'DPI - Department of Public Information', 'country': 'United States', 'city': 'New York, USA', 'link': 'https://unjobs.org/vacancies/1540347873232'}
2018-11-18 22:51:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540260978015>
{'job_title': 'Spring 2019 Energy, Water, and Sustainability Program Internship, Washington', 'organization': '\nStimson Center', 'country': 'United States', 'city': 'Washington, United States', 'link': 'https://unjobs.org/vacancies/1540260978015'}
2018-11-18 22:51:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540347780011>
{'job_title': 'Programme Assistant, Vi

...

2018-11-18 22:51:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/vacancies/1542153856157> (referer: https://unjobs.org/closing/4)
2018-11-18 22:51:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1542222909395>
{'job_title': 'Monitoring & Evaluation Associate, Cairo', 'organization': 'UNDP - United Nations Development Programme', 'country': 'Egypt', 'city': 'Cairo, Egypt', 'link': 'https://unjobs.org/vacancies/1542222909395'}
2018-11-18 22:51:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1542233059900>
{'job_title': 'Associate Protection and Neutrality Officer, Amman', 'organization': 'UNRWA - United Nations Relief and Works Agency for Palestine Refugees in the Near East', 'country': 'Jordan', 'city': 'Amman, Jordan', 'link': 'https://unjobs.org/vacancies/1542233059900'}
2018-11-18 22:51:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1542222922088>
{'job_title': 'PROJECT 

2018-11-18 22:51:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1541536400842>
{'job_title': 'Regional Finance Manager, Thornhill, Dumfries and Galloway, United Kingdom', 'organization': '\nThe Halo Trust (HALO)', 'country': 'United Kingdom', 'city': 'London, United Kingdom', 'link': 'https://unjobs.org/vacancies/1541536400842'}
2018-11-18 22:51:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1542153872481>
{'job_title': 'Information Management Officer, Port-au-Prince', 'organization': 'OCHA - Office for the Coordination of Humanitarian Affairs', 'country': 'Haiti', 'city': 'Port-au-Prince, Haiti', 'link': 'https://unjobs.org/vacancies/1542153872481'}
2018-11-18 22:51:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1541535770924>
{'job_title': 'Data Management Specialist, The Hague', 'organization': '\nNetherlands Development Finance Company (FMO)', 'country': 'Netherlands', 'city': 'The Hague

2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/19> (referer: None)
2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/23> (referer: None)
2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/25> (referer: None)
2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/24> (referer: None)
2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/27> (referer: None)
2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/28> (referer: None)
2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/26> (referer: None)
2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/29> (referer: None)
2018-11-18 22:51:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/cl

2018-11-18 22:51:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1539994945345>
{'job_title': 'Humanitarian-Development-Peace (HDP) Nexus support (Generic Call for the ExpRes Roster), Multiple', 'organization': 'UNDP - United Nations Development Programme', 'country': 'United States', 'city': 'New York, USA', 'link': 'https://unjobs.org/vacancies/1539994945345'}
2018-11-18 22:51:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://unjobs.org/vacancies/1540261294361>
{'job_title': 'Community Inclusion Adviser, Myanmar', 'organization': '\nVoluntary Service Overseas (VSO)', 'country': 'Myanmar', 'city': 'Yangon, Myanmar', 'link': 'https://unjobs.org/vacancies/1540261294361'}
2018-11-18 22:51:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/vacancies/1539851403395> (referer: https://unjobs.org/closing/19)
2018-11-18 22:51:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/vacancies/1539851402290> (referer: https://u

2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/89> (referer: None)
2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/90> (referer: None)
2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/88> (referer: None)
2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/91> (referer: None)
2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/92> (referer: None)
2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/93> (referer: None)
2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/95> (referer: None)
2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/closing/94> (referer: None)
2018-11-18 22:52:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://unjobs.org/cl

Finally I convert the json file to a panda dataframe and then I export it to Excel.

In [3]:
dfjson = pd.read_json('unjobs.json')
dfjson

,city,country,job_title,link,organization
0,"Jakarta, Indonesia",Indonesia,"Senior Officer Knowledge Management, Jakarta",https://unjobs.org/vacancies/1541601965788,ASEAN - Association of Southeast Asian Nations
1,"Frankfurt, Germany",Germany,Sachbearbeiter Organisation und Verwaltung von...,https://unjobs.org/vacancies/1542233965875,\nDeutsche Gesellschaft für Internationale Zus...
2,"Nairobi, Kenya",Kenya,Communications Intern with Government Practice...,https://unjobs.org/vacancies/1541941526126,World Wildlife Fund (WWF)
3,"Dhaka, Bangladesh",Bangladesh,Technical Officer - Fisheries (three positions...,https://unjobs.org/vacancies/1542228403649,WorldFish
4,"Bangkok, Thailand",Thailand,"Programme Officer, Bangkok, Thailand",https://unjobs.org/vacancies/1541703761933,ILO - International Labour Organization
5,"Washington, United States",United States,"Senior Associate, Business Development (Health...",https://unjobs.org/vacancies/1541539936834,\nPalladium Group
6,"Brasilia, Brazil",Brazil,BRA/06/024 - Contratação de consultor para o P...,https://unjobs.org/vacancies/1541601137803,UNDP - United Nations Development Programme
7,"Nairobi, Kenya",Kenya,Intern - Administration (Host Country Services...,https://unjobs.org/vacancies/1541546280421,UNON - United Nations Office at Nairobi
8,"Tripoli, Libya",Libya,"National Post - Programme Analyst, Sexual Repr...",https://unjobs.org/vacancies/1542227079342,UNFPA - United Nations Population Fund
9,"Dhaka, Bangladesh",Bangladesh,"National Building Safety Officer, Dhaka, Bangl...",https://unjobs.org/vacancies/1541703775384,ILO - International Labour Organization


In [4]:
writer = pd.ExcelWriter('unjobs.xlsx')
dfjson.to_excel(writer,'Sheet1')
writer.save()